# Calculation of United Atom Bilayer Thickness 

In [4]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
#import scipy.cluster.hierarchy
import scipy.spatial

In [5]:
traj=md.load('ua_symRho=ptseven-gmxsolvated.xtc',top='ua_symRho=ptseven-gmxsolvated.gro')
name='ua_symRho=ptseven-gmxsolvated'

In [6]:
topology=traj.topology
allhead=topology.select('name P8 or name P11')
water=topology.select('name O')

In [7]:
resnames=[atom.residue.name for atom in topology.atoms]

# 1: Sort Lipids into Upper/Lower Leaflet


In [9]:
#x,y coordinates of all for finding neighbors to determine local midplane z-value
# here, best to use xy radius of neighbors, since Voronoi across 2 leaflets could have complications

allheadxy=list([] for _ in xrange(traj.n_frames))
allheadz=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(allhead)):
        allheadi=allhead[i]
        allheadxy[nn].append(traj.xyz[nn][allheadi][0:2:1])
        allheadz[nn].append(traj.xyz[nn][allheadi][2])


In [10]:
# mdtraj can't handle neighbors for gro (rather than xtc) inputs, so do it myself
#cutoffsq = 2.4**2; #square of maximum xy distance for neighbors
cutoffsq=1.0**2
neigh=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    neigh[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allheadxy[nn])):
        xyi=allheadxy[nn][i]
        for j in range(len(allhead)-i-1): #not self, but will add for midplane finding
            xyj=allheadxy[nn][j+i+1]
            distsq=(xyi[0]-xyj[0])**2 + (xyi[1]-xyj[1])**2
            if (distsq < cutoffsq):
                neigh[nn][i].append(j+i+1)
                neigh[nn][j+i+1].append(i)
  

In [11]:
#new leaflet id method based on tilt angles; PO4-C4A or ROH-C1 (both +6 beads)
num_head=len(allhead)
tiltvectors=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    tiltvectors[nn]=list([] for _ in xrange(num_head)) #store vector roh-c1 or po4-c4a
    
    for i in range(num_head):
        tiltvectors[nn][i]=traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[i]+6]


In [12]:
# must fix periodic boundary condition errors in tiltvectors, then calculate tiltangle
norms=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    norms[nn]=list([] for _ in xrange(len(allhead)))
    halfx=0.5*traj.unitcell_lengths[nn][0]
    halfy=0.5*traj.unitcell_lengths[nn][1]
    halfz=0.5*traj.unitcell_lengths[nn][2]
    for i in range(len(allhead)):
        norms[nn][i]=np.linalg.norm(tiltvectors[nn][i])
        if (norms[nn][i] > halfz):
            if (np.abs(tiltvectors[nn][i][0]) > halfx):
                if (tiltvectors[nn][i][0]>0): 
                    tiltvectors[nn][i][0]=tiltvectors[nn][i][0]-2*halfx
                else:
                    tiltvectors[nn][i][0]=tiltvectors[nn][i][0]+2*halfx
            if (np.abs(tiltvectors[nn][i][1]) > halfy):
                if (tiltvectors[nn][i][1]>0): 
                    tiltvectors[nn][i][1]=tiltvectors[nn][i][1]-2*halfy
                else:
                    tiltvectors[nn][i][1]=tiltvectors[nn][i][1]+2*halfy
            if (np.abs(tiltvectors[nn][i][2]) > halfz):
                if (tiltvectors[nn][i][2]>0): 
                    tiltvectors[nn][i][2]=tiltvectors[nn][i][2]-2*halfz
                else:
                    tiltvectors[nn][i][2]=tiltvectors[nn][i][2]+2*halfz

    for i in range(len(allhead)):
        norms[nn][i]=np.linalg.norm(tiltvectors[nn][i])


In [13]:
# use neigh to find local average tilt vector, outliers are not in a leaflet
# if pointing neg in z, flip in x,y,z for making average in outer leaflet
localvector=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    localvector[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        localvectorsx=[]
        localvectorsy=[]
        localvectorsz=[]
        sgn=np.sign(tiltvectors[nn][i][2])
        localvectorsx.append(sgn*tiltvectors[nn][i][0]) #include self
        localvectorsy.append(sgn*tiltvectors[nn][i][1]) #include self
        localvectorsz.append(sgn*tiltvectors[nn][i][2]) #include self
        for j in range(len(neigh[nn][i])):
            sgn=np.sign(tiltvectors[nn][neigh[nn][i][j]][2])
            localvectorsx.append(sgn*tiltvectors[nn][neigh[nn][i][j]][0])
            localvectorsy.append(sgn*tiltvectors[nn][neigh[nn][i][j]][1])
            localvectorsz.append(sgn*tiltvectors[nn][neigh[nn][i][j]][2])
        localvector[nn][i]=[np.mean(localvectorsx),np.mean(localvectorsy),np.mean(localvectorsz)]
       

In [14]:
# find angle between orientation vector and local average orientation vector for each lipid
# in range [0,180]
diffangle=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    diffangle[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        normlv=np.linalg.norm(localvector[nn][i])
        normtv=np.linalg.norm(tiltvectors[nn][i])
        cos=np.dot(localvector[nn][i],tiltvectors[nn][i])/(normlv*normtv)
        if (cos==0):
            diffangle[nn][i]=90
        elif (cos==1 and np.sign(localvector[nn][i][2])==np.sign(tiltvectors[nn][i][2])):
            diffangle[nn][i]=0
        elif (cos==1 and np.sign(localvector[nn][i][2])==np.sign(tiltvectors[nn][i][2])):
            diffangle[nn][i]=180
        else:
            diffangle[nn][i]=np.arccos(cos)*180./np.pi


In [15]:
#have array with values placing each head in one leaflet: 0=lower, 1=upper, 2=between
head_leaflet=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    head_leaflet[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        if (diffangle[nn][i]>120):
            head_leaflet[nn][i]=0
        elif (diffangle[nn][i]<60):
            head_leaflet[nn][i]=1
        else:
            head_leaflet[nn][i]=2

In [16]:
# want to identify all midplane chol and remove all "midplane" phospholipids
# tilt angle insufficient
# chol: find distance from ROH to nearest PO4; if beyond threshold, then in midplane; 
# else, copy leaflet ID (do for all phospholipids); only if copying non-midplane

for nn in range(traj.n_frames):
    fixedchol=0
    fixedphos=0
    midplanechol=0
    for i in range(len(allhead)):
        mindist=100.0
        if (head_leaflet[nn][i]==2 and (not resnames[allhead[i]]=='CHOL')): #phospholipids, must assign
            for j in range(len(neigh[nn][i])):
                if (not resnames[allhead[neigh[nn][i][j]]]=='CHOL'):
                    dist=np.linalg.norm(traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[neigh[nn][i][j]]])
                    if (dist < mindist and (not head_leaflet[nn][neigh[nn][i][j]]==2)):
                        mindist=dist
                        correct_leaflet=head_leaflet[nn][neigh[nn][i][j]]
            head_leaflet[nn][i]=correct_leaflet
            fixedphos=fixedphos+1
    #must fix all phospholipids before all chols, since some may use fixed phospholipid leaflet id
    # so, must go through allhead twice
    for i in range(len(allhead)):
        mindist=100.0
        if (resnames[allhead[i]]=='CHOL'): #all chol, ignoring angle
            for j in range(len(neigh[nn][i])):
                if (not resnames[allhead[neigh[nn][i][j]]]=='CHOL'):
                    dist=np.linalg.norm(traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[neigh[nn][i][j]]])
                    if (dist < mindist and (not head_leaflet[nn][neigh[nn][i][j]]==2)):
                        mindist=dist
                        correct_leaflet=head_leaflet[nn][neigh[nn][i][j]]
            if (mindist<1.4): #close to a headgroup, so in a leaflet
                head_leaflet[nn][i]=correct_leaflet
                fixedchol=fixedchol+1
            else:
                head_leaflet[nn][i]=2
                midplanechol=midplanechol+1

In [17]:
# only need upper leaflet heads for this situation
upperheads=list([] for _ in xrange(traj.n_frames))
lowerheads=list([] for _ in xrange(traj.n_frames))
middle=list([] for _ in xrange(traj.n_frames))
waterhead=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
#    upperheads[nn]=[]
    for i in range(len(allhead)):
        if head_leaflet[nn][i]==1:
            upperheads[nn].append(allhead[i])
        if head_leaflet[nn][i]==0:
            lowerheads[nn].append(allhead[i])
        if head_leaflet[nn][i]==2:
            middle[nn].append(allhead[i])

for nn in range(traj.n_frames): 
    for i in range(len(water)): 
        waterhead[nn].append(water[i])

In [18]:
upperheadxy=list([] for _ in xrange(traj.n_frames)) #[frame][head]
for nn in range(traj.n_frames):
#    headxy[nn]=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        upperheadsi=upperheads[nn][i]
        upperheadxy[nn].append(traj.xyz[nn][upperheadsi][0:3:1])

lowerheadxy = list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames): 
    for i in range(len(lowerheads[nn])): 
        lowerheadsi=lowerheads[nn][i]
        lowerheadxy[nn].append(traj.xyz[nn][lowerheadsi][0:3:1])
        

# 2: Find Lipid Opposite Leaflet Neighbors

In [19]:
#find closest neighbors between upperheads and lowerheads 
#lipid_opp_distance = list([] for _ in xrange(traj.n_frames))
lipid_opp_neighbors = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames): 
    for i in range(len(upperheadxy[nn])): 
        pairs=[]
        for j in range(len(lowerheadxy[nn])): 
            r=(upperheadxy[nn][i][0]-lowerheadxy[nn][j][0])**2 + (upperheadxy[nn][i][1]-lowerheadxy[nn][j][1])**2
            if r<5:
                pairs.append(lowerheads[nn][j])
        lipid_opp_neighbors[nn].append([i,pairs])
        

In [20]:
#find closest neighbors between upperheads and lowerheads 
#lipid_opp_distance = list([] for _ in xrange(traj.n_frames))
lipid_opp_neighbors_rev = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames): 
    for i in range(len(lowerheadxy[nn])): 
        pairs=[]
        for j in range(len(upperheadxy[nn])): 
            r=(upperheadxy[nn][j][0]-lowerheadxy[nn][i][0])**2 + (upperheadxy[nn][j][1]-lowerheadxy[nn][i][1])**2
            if r<5:
                pairs.append(upperheads[nn][j])
        lipid_opp_neighbors_rev[nn].append([i,pairs])
        

In [21]:
#sort through the list of close neighbors and find closest lipid
lipid_pairs = list([] for _ in xrange(traj.n_frames))
lipid_distance= list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames): 
    for i in range(len(upperheads[nn])): 
        lipid_distance[nn].append(10000000)
        lipid_pairs[nn].append([i,0])
        
        
for nn in range(traj.n_frames): 
    for i in range(len(lipid_opp_neighbors[nn])):
        for j in range(len(lipid_opp_neighbors[nn][i][1])):
            upperindex=lipid_opp_neighbors[nn][i][0]
            lowerheadvalue=lipid_opp_neighbors[nn][i][1][j]
            lowerindex=lowerheads[nn].index(lowerheadvalue)
            d = (upperheadxy[nn][upperindex][0] - lowerheadxy[nn][lowerindex][0])**2 + (upperheadxy[nn][upperindex][1] - lowerheadxy[nn][lowerindex][1])**2 + (upperheadxy[nn][upperindex][2] - lowerheadxy[nn][lowerindex][2])**2
           
            if d<lipid_distance[nn][i]: 
                #print('true')
                lipid_distance[nn][i] = d 
                lipid_pairs[nn][i] = [upperindex,lowerindex]